In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 5.0 MB/s 
     |████████████████████████████████| 596 kB 62.0 MB/s 
     |████████████████████████████████| 6.6 MB 70.6 MB/s 
     |████████████████████████████████| 101 kB 13.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch

In [ ]:
import numpy as np
import pandas as pd
import re
import random


from tqdm.notebook import tqdm
import transformers

if torch.cuda.is_available():    
    device = torch.device("cuda")
else:
    device = torch.device("cpu")


In [ ]:
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('sberbank-ai/rugpt3medium_based_on_gpt2')

In [ ]:
from transformers import GPT2LMHeadModel, AdamW

model = GPT2LMHeadModel.from_pretrained(
    'sberbank-ai/rugpt3medium_based_on_gpt2',
    output_attentions = False,
    output_hidden_states = False,
)

model.to(device);

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr = 1e-5, eps = 1e-8)

Здесь загружаем веса в модель ⬇

In [ ]:
model.load_state_dict(torch.load("/content/drive/MyDrive/lost/lost_state_dict"))
model.eval();

In [ ]:
def generate(prompt, len_gen=20, temperature=0.8):
    generated = tokenizer.encode(prompt)
    context = torch.tensor([generated]).to(device)
    past = None

    for i in tqdm(range(len_gen)):
        output, past = model(context, past_key_values=past).values()
        # token = torch.argmax(output[..., -1, :], dim=-1)
        output = output / temperature
        token = torch.distributions.Categorical(logits=output[..., -1, :]).sample()
        
        generated += token.tolist()
        context = token.unsqueeze(0)

    sequence = tokenizer.decode(generated)

    return sequence

In [ ]:
generate('я люблю тебя', len_gen=120, temperature=0.9)

  0%|          | 0/120 [00:00<?, ?it/s]

'я люблю тебя, Москва, я так скучал!"  Кстати, в обоих случаях – и в первом и во втором – мощная личность, яркая личность с артистическими задатками."\n"В феврале Илон Маск продал акций Твиттера на рекордную сумму в $6.5 миллиарда. За первую неделю марта рыночная стоимость бумаг скакнула почти в 2 раза! При этом Илон не скрывал, что не контролирует текущую деятельность компании и что существует продажа мажоритарной доли Оксане Горбуновой. Только вплоть до последнего времени Илон публично опровергал информацию о сделке."\n"'

In [ ]:
torch.cuda.empty_cache()